In [ ]:
import csv
import cv2
import numpy as np # numpy array is the format required by Keras

lines = []
with open('data/driving_log.csv') as csvfile:
    reader =csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = [] # this represents the features
measurements = [] # this represents the labels to predict

for line in lines[1:]: #skipping the headers
    
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path ='/data/IMG'+ filename
    image = cv2.imread(current_path)
    images.append(image)
    
    # extracting the steering wheel as labels
    #print ("steering angle", line[3])
    measurement = float(line[3])
    measurements.append(measurements)
    
# Data to feed the algorithm
X_train = np.array(images)
y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense

#Testing the data on a simplistic neural network
model = Sequential()
model.add(Flatten(input_shape=(160,320,3)))
model.add(Dense(1))

# it is a regression network not a classification network, 'measn square errors' is used
model.compile(loss='mse', optimizer='adam')
# epoch is set to 7 because after this number the prediction accuracy decreases
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=7)

model.save('model.h5')
